In [ ]:
import torch
from torch_geometric.data import Data

import torch.nn as nn
import torch_geometric.nn as pyg_nn

import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx
from torch_geometric.nn import GCNConv

import numpy as np

In [3]:
print(torch.cuda.get_device_name(torch.cuda.current_device()))  # GPU name

# CUDA support
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

NVIDIA GeForce GTX 1650
Using device: cuda


In [ ]:
class PINN_GCN(torch.nn.Module):
    """
    Physics-Informed Graph Convolutional Network

    - 5 GCN layers
    - Tanh activations after each except the last
    """
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=5):
        super(PINN_GCN, self).__init__()
        assert num_layers >= 2, "num_layers must be at least 2"
        self.convs = torch.nn.ModuleList()

        # Input layer
        self.convs.append(GCNConv(in_channels, hidden_channels))
        # Hidden layers
        for _ in range(num_layers - 2):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
        # Output layer
        self.convs.append(GCNConv(hidden_channels, out_channels))

    def forward(self, x, edge_index):
        """
        x: node features (e.g., coordinates, boundary flags, etc.)
        edge_index: graph connectivity
        """
        for conv in self.convs[:-1]:
            x = conv(x, edge_index)
            x = torch.tanh(x)
        # Last layer without activation
        x = self.convs[-1](x, edge_index)
        return x